# THƯ VIỆN CẦN THIẾT

In [98]:
import pandas as pd
import numpy as np

# PHÂN TÍCH DỮ LIỆU
- [INSERT MÔ TẢ SƠ BỘ VỀ 3 FILE/CÁC CÂU HỎI/CÁC BƯỚC LÀM]

## I. TIỀN XỬ LÝ DỮ LIỆU
### Dữ liệu có bao nhiêu dòng, bao nhiêu cột?

In [99]:
df_playlist = pd.read_csv("data/playlist.csv")
df_track = pd.read_csv("data/track.csv")
df_user = pd.read_csv("data/user.csv")

df_playlist.shape, df_track.shape, df_user.shape

((3372, 5), (46815, 9), (40129, 5))

### Các file dữ liệu có các dòng nào bị trùng lắp hoàn toàn không?
- Thấy rằng từ output, không có dòng nào bị trùng lắp hoàn toàn trong từng file dữ liệu

In [100]:
df_playlist.duplicated().sum(), df_track.duplicated().sum(), df_user.duplicated().sum()

(0, 0, 0)

### Mỗi cột thuộc kiểu dữ liệu nào? Có cột nào bị thiếu dữ liệu không?
- Với mỗi file, kết hợp 2 công việc này vào 1 dataframe. Dataframe này gồm các cột chính là các cột thuộc tính trong lần lượt 3 file x 2 hàng có tên: 
    - `DataType` - thể hiện kiểu dữ liệu của các thuộc tính trong file
    - `NaN counts` - đếm số lượng dữ liệu thiếu tại mỗi cột  
- Theo đồ án 1, ý nghĩa của mỗi thuộc tính trong mỗi file như sau:
    - `playlist.csv`:
        - `id`: mã của playlist đó, mã này dùng để phân biệt với các playlist khác, mỗi playlist sẽ có một mã số `id` duy nhất.
        - `name`: là tên của playlist đó (trong trường hợp playlist đó không có tên thì ta gán `name` là `null`).
        - `genre`: là thể loại của playlist đó (trong trường hợp playlist đó chưa được đặt thể loại thì ta gán `genre` là `null`).
        - `likes_count`: tổng số lượt likes của playlist
        - `track_ids`: là chuỗi gồm danh sách các track có track_id nằm trong playlist đó.
    - `track.csv`:
        - `id`: id của track. Lưu ý 1 track có thể nằm trong nhiều playlist
        - `name`: tên track
        - `genre`: thể loại track
        - `playlist_id`: playlist_id mà track có mã là `id` thuộc về
        - `author_id`: mỗi track có 1 user có mã đại diện là `author_id` phụ trách
        - `likes_count`: tổng số lượt likes của track
        - `playback_count`: tổng số lượt replay của track
        - `download_count`: tổng số lượt tải về track đó
        - `created_date`: ngày track đó được tạo
    - `user.csv`:
        - `id`: mã người dùng, là duy nhất. Lưu ý 1 người dùng có thể phụ trách 1 hoặc nhiều track
        - `full_name`: họ và tên user
        - `username`: tên tài khoản user
        - `track_id`: track mà user 
        - `followers_count`: số lượng người theo dõi user đó

In [101]:
pd.DataFrame([df_playlist.dtypes, df_playlist.isna().sum()], index= ['DataType', 'NaN counts'])

,playlist_id,playlist_name,genre,likes_count,track_ids
DataType,int64,object,object,int64,object
NaN counts,0,0,2706,0,38


In [102]:
pd.DataFrame([df_track.dtypes, df_track.isna().sum()], index= ['DataType', 'NaN counts'])

,track_id,track_name,track_genre,playlist_id,author_id,likes_count,playback_count,download_count,created_date
DataType,int64,object,object,int64,int64,float64,float64,float64,object
NaN counts,0,0,6623,0,0,882,882,882,0


In [103]:
pd.DataFrame([df_user.dtypes, df_user.isna().sum()], index= ['DataType', 'NaN counts'])

,user_id,full_name,username,track_id,followers_count
DataType,int64,object,object,int64,int64
NaN counts,0,15728,0,0,0


### Loại các dòng bị thiếu dữ liệu 


-  File `playlist.csv`: từ mô tả thuộc tính và kiểu dữ liệu của `track_ids`, thì cần loại đi các playlist không có track nào (tức thuộc tính `track_ids` bị thiếu). Ngoài ra, cũng có 2706 dòng bị thiếu thuộc tính `genre` nhưng qua quan sát sơ bộ file `playlist.csv`, hầu hết tên các playlist (`playlist_name`) sẽ được đặt theo tên thể loại nhạc nên ta sẽ không xóa các dòng thiếu thuộc tính `genre`

In [104]:
df_playlist = df_playlist[df_playlist['track_ids'].notna()]
df_playlist.shape

(3334, 5)

- File `track.csv`: tương tự với cách giải thích của `playlist.csv`, ta sẽ không xóa các dòng thiếu thuộc tính `track_genre`. Tuy nhiên sẽ xóa các dòng thiếu thuộc tính likes_count, playback_count, download_count vì đây là các thuộc tính numeric quan trọng cho bước khám phá dữ liệu sau này

In [105]:
df_track = df_track[(df_track['likes_count'].notna()) & (df_track['playback_count'].notna()) & (df_track['download_count'].notna())]
df_track.shape

(45933, 9)

- File `user.csv`: không xóa dòng nào vì tuy các user có thể không có tên `full_name` nhưng ta vẫn có thể thao tác với dataframe df_user dựa vào định danh của user đó (tức `user_id`) 

### Chuyển đổi kiểu dữ liệu cho phù hợp


- File `playlist.csv`: từ mô tả thuộc tính và kiểu dữ liệu của `track_ids`, thì cần đổi kiểu dữ liệu của thuộc tính `track_ids` từ **string** sang dạng **list các số nguyên**

In [106]:
df_playlist['track_ids'] = df_playlist['track_ids'].apply(lambda x: [int(item) for item in x.split(',')])

- File `track.csv`: 
    - Cần đổi kiểu dữ liệu của 2 cột `likes_count`, `playback_count`từ **float** sang **int**
    - Cần đổi kiểu dữ liệu của `created_date` sang kiểu **datetime**

In [110]:
df_track[['likes_count', 'playback_count', 'download_count']] = df_track[['likes_count', 'playback_count', 'download_count']].astype(int)
df_track['created_date'] = pd.to_datetime(df_track['created_date'])
df_track[['likes_count', 'playback_count', 'download_count', 'created_date']].head()

,likes_count,playback_count,download_count,created_date
0,147202,5715630,0,2021-10-01 21:15:32+00:00
1,61082,2316084,0,2021-10-07 01:59:52+00:00
2,55067,1971749,0,2021-10-19 16:25:03+00:00
3,2190,49610,0,2021-10-29 03:44:10+00:00
4,46192,1678541,0,2021-10-15 05:48:04+00:00


### Kiểm tra ngoại lệ

In [108]:
# SOMETHING HERE

# II. ĐƯA RA CÂU HỎI CÓ Ý NGHĨA CẦN TRẢ LỜI

## 1. Phân bố số lượng playlist cho các thể loại

## 2. Phân bố số lượng track cho các playlist -> hist